In [2]:
pip install playsound

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install cmake

     |████████████████████████████████| 74.6 MB 47.6 MB/s eta 0:00:01   |███▉                            | 9.0 MB 6.5 MB/s eta 0:00:11     |██████████████████████▌         | 52.6 MB 11.4 MB/s eta 0:00:02
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install dlib

  Using cached dlib-19.23.1.tar.gz (7.4 MB)
  Created wheel for dlib: filename=dlib-19.23.1-cp39-cp39-macosx_10_15_x86_64.whl size=3688909 sha256=4fb789e950e72b6b200e41594a14e770ca625eb3eb991bb7fa8f50c6fa78a5ef
  Stored in directory: /Users/daniajuvale/Library/Caches/pip/wheels/5b/91/67/506f5d51996943adad6ce0924f32514fa060fb66eda000dfd2
Successfully built dlib
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import the necessary packages
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import imutils
import time
import dlib
import cv2

# calculate lip distance
def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))
    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))
    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)
    distance = abs(top_mean[1] - low_mean[1])
    return distance

# E vowel threshold
E_LT = 9.0
E_UT = 12.0

# initialize dlib's face detector (HOG-based) and then create the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# start the video stream thread
print("[INFO] starting video stream thread...")
vs = VideoStream(src=0).start()
time.sleep(1.0)
cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)

while True:
	# grab the frame from the threaded video file stream, resize it, and convert it to grayscale channels)
	frame = vs.read()
	frame = imutils.resize(frame, width=450)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	# detect faces in the grayscale frame
	rects = detector(gray, 0)
 	# loop over the face detections
	for rect in rects:
		# determine the facial landmarks for the face region, then convert the facial landmark (x, y)-coordinates to a NumPy array
		shape = predictor(gray, rect)
		shape = face_utils.shape_to_np(shape)

		# calculate lip distance 
		distance = lip_distance(shape)

		# draw lip contours
		lip = shape[48:60]
		cv2.drawContours(frame, [lip], -1, (0, 255, 0), 1)

			
		# thresholds and frame counters
		if E_LT < distance < E_UT:
		    cv2.putText(frame, "E detected: {:.2f}".format(distance), (300, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
 
	# show the frame
	cv2.imshow("Frame", frame)
	cv2.resizeWindow('Frame', 600,600)
	key = cv2.waitKey(1) & 0xFF
 
	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

# command line:
# python detect_drowsiness.py --shape-predictor shape_predictor_68_face_landmarks.dat
# python detect_drowsiness.py --shape-predictor shape_predictor_68_face_landmarks.dat --alarm alarm.wav

[INFO] loading facial landmark predictor...
[INFO] starting video stream thread...
